In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

project_dir = r'C:\Users\DELL\All Files\Personal\Machine Learning\Fractal 3\charts\charts'
csv_file = 'train_val.csv'

# load the csv file
df = pd.read_csv(f'{project_dir}\\{csv_file}')
df['image_index']=df['image_index'].astype(str)
df['image_index']=df['image_index'].apply(lambda x:x+'.png')
# define the path to the image folders
train_path = 'charts/train_val/'
test_path = 'charts/test/'

# split the data into training and validation sets
train_df = df.sample(frac=0.8,random_state=42)
val_df = df.drop(train_df.index)

# create data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_path,
        x_col="image_index",
        y_col="type",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
        dataframe=val_df,
        directory=train_path,
        x_col="image_index",
        y_col="type",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')


Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


C:\Users\DELL\anaconda3\lib\site-packages\keras\preprocessing\image.py:1137: UserWarning: Found 800 invalid image filename(s) in x_col="image_index". These filename(s) will be ignored.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\keras\preprocessing\image.py:1137: UserWarning: Found 200 invalid image filename(s) in x_col="image_index". These filename(s) will be ignored.
  warnings.warn(


In [3]:
from tensorflow.keras.applications import AlexNet

# load the pre-trained model
base_model = AlexNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# add our own classifier on top
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))

# freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# train the model
history = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=10,
        validation_data=val_generator,
        validation_steps=len(val_generator))

# plot the loss
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()


ImportError: cannot import name 'AlexNet' from 'tensorflow.keras.applications' (C:\Users\DELL\anaconda3\lib\site-packages\keras\api\_v2\keras\applications\__init__.py)